In [1]:
from cxr_foundation.inference import generate_embeddings, InputFileType, OutputFileType, ModelVersion
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import logging
from PIL import Image

2023-11-14 18:41:33.636073: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 18:41:33.684855: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 18:41:33.684879: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 18:41:33.684909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 18:41:33.692203: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-14 18:41:33.692629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [6]:
import os
from imageio import imread
import numpy as np
from tqdm import tqdm

def open_image_with_imopen(image_path):
    try:
        # Open the image using imopen
        image_data = imread(image_path)

        # Convert image to NumPy array if needed
        if isinstance(image_data, np.ndarray):
            return image_data.astype(np.float32)
        else:
            return None
    except Exception as e:
        return None

def process_images(directory):
    # List to store paths of images that couldn't be opened
    failed_images = []

    # Get the list of image files
    image_files = [filename for filename in os.listdir(directory) if filename.endswith(('.jpg', '.jpeg', '.png'))]

    # Iterate through files with tqdm for a progress bar
    for filename in tqdm(image_files, desc="Processing Images", unit="image"):
        image_path = os.path.join(directory, filename)

        # Try opening the image with imopen
        if open_image_with_imopen(image_path) is None:
            failed_images.append(image_path)

    return failed_images

# Example usage
directory_path = "/data4/CheXpert/preproc_224x224"
failed_image_paths = process_images(directory_path)

if not failed_image_paths:
    print("All images were successfully opened.")
else:
    print("Failed to open the following images:")
    print(len(failed_image_paths))


Processing Images:   0%|          | 0/127118 [00:00<?, ?image/s]/tmp/ipykernel_33641/3980639581.py:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image_data = imread(image_path)
Processing Images: 100%|██████████| 127118/127118 [06:22<00:00, 332.57image/s]

Failed to open the following images:
84


In [7]:
len(failed_image_paths)

84

In [8]:
def export_to_file(file_path, data):
    with open(file_path, 'w') as file:
        for item in data:
            file.write("%s\n" % item)

export_file_path = "failed_images.txt"
export_to_file(export_file_path, failed_image_paths)
print(f"Failed image paths exported to {export_file_path}.")

Failed image paths exported to failed_images.txt.


In [2]:
EMBEDDINGS_DIR = "/data4/CheXpert/chexpert_embedding_normal"
MODEL_VERSION = ModelVersion.V1
data_dir = '../datafiles/chexpert/'
img_data_dir = '/data4/CheXpert/'

In [3]:
df_test = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test['Path'] = df_test['Path'].apply(lambda x: img_data_dir + x)

In [4]:
first_image = Image.open(df_test.iloc[2, 0])
print(first_image.size)

(2828, 2320)


In [5]:
df_img_data = pd.read_csv(data_dir + 'chexpert.sample.csv')
df_img_data['path_preproc'] = df_img_data['path_preproc'].apply(lambda x: img_data_dir + x)

In [6]:
import logging
logging.getLogger('google.auth._default').setLevel(logging.ERROR)

In [8]:
generate_embeddings(input_files=df_img_data["path_preproc"].values, output_dir=EMBEDDINGS_DIR,
    input_type=InputFileType.PNG, output_type=OutputFileType.NPZ, model_version=MODEL_VERSION)

In [9]:
df_img_data["Path"].values[:5]

array(['/data4/CheXpert/CheXpert-v1.0/train/patient00002/study2/view1_frontal.jpg',
       '/data4/CheXpert/CheXpert-v1.0/train/patient00002/study1/view1_frontal.jpg',
       '/data4/CheXpert/CheXpert-v1.0/train/patient00003/study1/view1_frontal.jpg',
       '/data4/CheXpert/CheXpert-v1.0/train/patient00004/study1/view1_frontal.jpg',
       '/data4/CheXpert/CheXpert-v1.0/train/patient00005/study1/view1_frontal.jpg'],
      dtype=object)

In [5]:
first_image = Image.open(df_img_data.iloc[0, 0])
first_image.save('test.png', format='PNG')

In [6]:
df = pd.DataFrame({'Path': ['test.png']})

In [7]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Generate and store a few embeddings in .npz format
generate_embeddings(input_files=df["Path"].values, output_dir=EMBEDDINGS_DIR,
    input_type=InputFileType.PNG, output_type=OutputFileType.NPZ, model_version=MODEL_VERSION)

/home/tsounack/miniconda3/envs/chexploration/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
INFO:root:Successfully generated '/data4/CheXpert/chexpert_embedding_normal/test.npz'
